# 1. 유명한 데이터 추리기

In [23]:
import pandas as pd
import xml.etree.ElementTree as ET
from pathlib import Path
import os
from tqdm import tqdm
import shutil

DATA_PATH = "../csv_files"
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_row', None)

In [2]:
products_df = pd.read_csv(os.path.join(DATA_PATH,"products.csv"), encoding="utf-8")
category_df = pd.read_csv(os.path.join(DATA_PATH,"category.csv"), encoding="utf-8")
products_to_disliked_df = pd.read_csv(os.path.join(DATA_PATH,"products_to_disliked.csv"), encoding="utf-8")
all_df = pd.merge(products_df, category_df, left_on="category_id", right_on="id", how="inner")

In [11]:
available_products = set(products_to_disliked_df['product_id'])

In [18]:
snack_df = all_df[all_df['cat_1']=='과자'].drop(columns=['id_y']).rename(columns={"id_x":"id"})
snack_filtered_df = snack_df[snack_df['id'].isin(available_products)]
snack_filtered_df = snack_filtered_df[snack_filtered_df['cat_2']!='껌']
#add = snack_df[snack_df['id'].isin({1029,1445,669,985,961,979,758,785,996,968,989,724,837,838})]
#snack_filtered_df = pd.concat([snack_filtered_df , add])
#snack_filtered_df = snack_filtered_df.drop([1165])

#print(snack_filtered_df.shape)
#display(snack_filtered_df)

# 2. 이미지 데이터

1) annotations 폴더를 돌아다니면서, 해당하지 않는 데이터의 폴더는 지워버림
2) 해당하는 데이터를 만나면 checked true로 바꿈

## Train

In [2]:
train_list_df = pd.read_csv(os.path.join(DATA_PATH, "train_list.csv"), encoding="utf-8")
train_list_df.head(3)

,product_id,category_id,item_no,barcode,name,comp_nm,cat_1,cat_2,cat_3,cat_4
0,614,49,15049,8801037089059,동서 오레오 초콜릿 샌드위치 쿠키 100G,동서식품,과자,비스킷,샌드,샌드
1,704,50,25029,8801019314759,해태 사브레 252G,해태,과자,비스킷,쿠키,쿠키
2,707,50,25043,8801117277611,오리온 다이제 초코 312G,오리온,과자,비스킷,쿠키,쿠키


In [8]:
DATA_ANNO_PATH = "D:/dataset/상품 이미지/Training/annotations"
checked = [False for _ in range(0, 100)]
print(DATA_ANNO_PATH)

D:/dataset/상품 이미지/Training/annotations


In [16]:
dirs = os.scandir(DATA_ANNO_PATH)
dir_list = []
for dir in dirs:
    if ".zip" in dir.name:
        continue
    dir_list.append(dir.name)
print(dir_list)

['[라벨]과자1', '[라벨]과자2', '[라벨]과자3', '[라벨]과자4', '[라벨]디저트', '[라벨]면류', '[라벨]상온HMR1', '[라벨]상온HMR2', '[라벨]상온HMR3', '[라벨]소스1', '[라벨]소스2', '[라벨]음료1', '[라벨]음료2', '[라벨]음료3', '[라벨]통조림_안주']


In [20]:
item_no_list = list(train_list_df['item_no'])
item_no_set = set(item_no_list)
item_no_dict = dict()
for i in range(0, 100):
    item_no_dict[item_no_list[i]] = i

In [24]:
for dir in dir_list: # [라벨]~
    subpath = os.path.join(DATA_ANNO_PATH, dir) # 상품아이템
    subdirs = os.scandir(subpath)

    for subdir in subdirs:
        i_no = subdir.name.split("_")[0]
        if i_no in item_no_set:
            idx = item_no_dict[i_no]
            checked[idx] = True
            continue
        
        shutil.rmtree(subdir)

In [25]:
for i in range(0, 100):
    if not checked[i]:
        print(i) # 어노테이션 다 있음

## Validation

In [27]:
DATA_ANNO_PATH_V = "D:/dataset/상품 이미지/Validation/annotations"
item_no_list_v = list(train_list_df['item_no'])
item_no_set_v = set(item_no_list)
item_no_dict_v = dict()
checked_v = [False for _ in range(0, 100)]
for i in range(0, 100):
    item_no_dict_v[item_no_list_v[i]] = i

In [28]:
dirs = os.scandir(DATA_ANNO_PATH_V)
dir_list = []
for dir in dirs:
    if ".zip" in dir.name:
        continue
    dir_list.append(dir.name)
print(dir_list)

['[라벨]과자', '[라벨]면류', '[라벨]상온HMR', '[라벨]소스', '[라벨]음료']


In [29]:
for dir in dir_list: # [라벨]~
    subpath = os.path.join(DATA_ANNO_PATH_V, dir) # 상품아이템
    subdirs = os.scandir(subpath)

    for subdir in subdirs:
        i_no = subdir.name.split("_")[0]
        if i_no in item_no_set_v:
            idx = item_no_dict_v[i_no]
            checked_v[idx] = True
            continue
        
        shutil.rmtree(subdir)

In [30]:
for i in range(0, 100):
    if not checked_v[i]:
        print(i) # 어노테이션 다 있음